## Разметка и извлечение именованных сущностей

### Разметка слов с помощью частей речи (Parts-Of-Speech)

<img src='image/pos_tagging.PNG'>

POS разметка - определение части речи и грамматических характеристик слов в тексте (корпусе) с приписыванием им соответствующих тегов. POS tagging является одним из первых этапов компьютерного анализа текста. Эта задача не простая, так как конкретное слово может иметь различную часть речи в зависимости от контекста, в котором оно используется.
Например: в предложении “Дай мне свой ответ” ответ - это существительное, а в предложении “ответь на вопрос” ответ - это глагол.

POS разметка полезна для построения деревьев синтаксического анализа, которые используются при построении NER (именованных сущностей) и извлечении отношений между словами. POS-маркировка также необходима для построения лемматизаторов.

In [1]:
#nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

#### Универсальный набор тегов части речи

Ниже приведена таблица POS-тэгов: обозначение, часть речи, пример. Цветом выделены популярные части речи: существительное, глагол и вопросительное слово.

<img src='image/pos_tag_samples.jpg'>

Выведим примеры тэгов разных частей речи.

In [3]:
nltk.download('tagsets')

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [4]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


#### Сравнение POS тэггеров

POS-тэггер обрабатывает последовательность слов и определяет тэг части речи для каждого слова. Сравним работу нескольких тэггеров библиотеки nltk.tag. Проверять работоспособность теггеров будем на корпусе nltk.corpus.brown.

Отобразим распределение тэгов в корпусе brown. Можем видеть, что тэг "NN" наиболее популярный.

Корпус будет поделен на train и test, т.к. некоторым тэггерам необходимо обучение. Разметку визуально будем оценивать на примере test_sent, тестового предложения.

In [5]:
import nltk

In [6]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [7]:
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

tags = [tag for (word, tag) in
brown.tagged_words(categories='news')]
nltk.FreqDist(tags)

FreqDist({'NN': 13162, 'IN': 10616, 'AT': 8893, 'NP': 6866, ',': 5133, 'NNS': 5066, '.': 4452, 'JJ': 4392, 'CC': 2664, 'VBD': 2524, ...})

In [8]:
brown_tagged_sents = brown.tagged_sents(categories='news')
train_data = brown_tagged_sents[:int(len(brown_tagged_sents) * 0.9)]
test_data = brown_tagged_sents[int(len(brown_tagged_sents) * 0.9):]
test_sent = brown.sents(categories='news')[0]
test_data[0]

[('But', 'CC'),
 ('in', 'IN'),
 ('all', 'ABN'),
 ('its', 'PP$'),
 ('175', 'CD'),
 ('years', 'NNS'),
 (',', ','),
 ('not', '*'),
 ('a', 'AT'),
 ('single', 'AP'),
 ('Negro', 'NP'),
 ('student', 'NN'),
 ('has', 'HVZ'),
 ('entered', 'VBN'),
 ('its', 'PP$'),
 ('classrooms', 'NNS'),
 ('.', '.')]

#### DefaultTagger

Очень наивный тэггер, в данном случае присваивает тэг "NN" (noun) всем словам в тексте. Т.к. в английском тексте примерно 13% сущесвительных, то получим точность тэггирования примерно 0,13.

У каждого теггера есть метод .tag(), который принимает список токенов (обычно список слов, созданных токенизатором слов), где каждый токен-это одно слово.
Метод .evaluate() оценивает точность работы тэггера.

In [9]:
default_tagger = nltk.DefaultTagger('abracadabra')
display(default_tagger.tag(test_sent), default_tagger.evaluate(test_data))

[('The', 'abracadabra'),
 ('Fulton', 'abracadabra'),
 ('County', 'abracadabra'),
 ('Grand', 'abracadabra'),
 ('Jury', 'abracadabra'),
 ('said', 'abracadabra'),
 ('Friday', 'abracadabra'),
 ('an', 'abracadabra'),
 ('investigation', 'abracadabra'),
 ('of', 'abracadabra'),
 ("Atlanta's", 'abracadabra'),
 ('recent', 'abracadabra'),
 ('primary', 'abracadabra'),
 ('election', 'abracadabra'),
 ('produced', 'abracadabra'),
 ('``', 'abracadabra'),
 ('no', 'abracadabra'),
 ('evidence', 'abracadabra'),
 ("''", 'abracadabra'),
 ('that', 'abracadabra'),
 ('any', 'abracadabra'),
 ('irregularities', 'abracadabra'),
 ('took', 'abracadabra'),
 ('place', 'abracadabra'),
 ('.', 'abracadabra')]

0.0

#### UnigramTagger

UnigramTagger учитывает условную частоту тегов и предсказывает наиболее частый тег для каждого токена, не ориентируется на соседние слова.

In [10]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

0.8121200039868434

#### BigramTagger

BigramTagger будет учитывает тэги двух слов: текущее и предыдущее слово. Точность немного выше, чем у Unigram tagger.

In [11]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

0.8210904016744742

#### TrigramTagger

In [12]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

0.8185986245390212

#### RegexpTagger

RegexpTagger работает на основе поиска совпаднения с регулярными выражениями. Пример: числа можно сопоставить с регулярным выражением "\d" для присвоения тега CD (Cardinal number) или можно сопоставить известные шаблоны слов, такие как суффикс '.*ing$'.

In [13]:
patterns = [
    (r'.*ing$', 'VBG'),                # gerunds
    (r'.*ed$', 'VBD'),                 # simple past
    (r'.*es$', 'VBZ'),                 # 3rd singular present
    (r'.*ould$', 'MD'),                # modals
    (r'.*\'s$', 'NN$'),                # possessive nouns
    (r'.*s$', 'NNS'),                  # plural nouns
    (r'^-?[0-9]+(\.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'.*', 'NN'),                      # nouns (default) 
    (r'.*ment$', 'NN'),                # i.e. wonderment 
    (r'.*ful$', 'JJ')                  # i.e. wonderful 
]
regexp_tagger = RegexpTagger(patterns)
display(regexp_tagger.tag(test_sent), regexp_tagger.evaluate(test_data))

[('The', 'NN'),
 ('Fulton', 'NN'),
 ('County', 'NN'),
 ('Grand', 'NN'),
 ('Jury', 'NN'),
 ('said', 'NN'),
 ('Friday', 'NN'),
 ('an', 'NN'),
 ('investigation', 'NN'),
 ('of', 'NN'),
 ("Atlanta's", 'NN$'),
 ('recent', 'NN'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', 'NN'),
 ('no', 'NN'),
 ('evidence', 'NN'),
 ("''", 'NN'),
 ('that', 'NN'),
 ('any', 'NN'),
 ('irregularities', 'VBZ'),
 ('took', 'NN'),
 ('place', 'NN'),
 ('.', 'NN')]

0.20253164556962025

Судя по результату, регулярных выражений в patterns не достаточно, чтобы покрыть большой процент слов корпуса.

#### Комбинация тэггеров

Примущество Backoff Tagging в том, что если текущий тэггер не знает, как тэггировать слово, он передает это следующему и так далее, пока не пройдет перебор по всем тэггерам. В данному случае тэггирование производит последовательность UnigramTagger, BigramTagger, TrigramTagger. Комбинация тэггеров дала немного лучший результат, чем UnigramTagger, BigramTagger по отдельности.

In [14]:
from nltk.tag import TrigramTagger 

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data) 

0.843317053722715

#### Создание тэггера имен 

Подходим к теме именнованых сущностей (NER). Мы можем самостоятельно написать необходимый тэггер. Допустим, мы хотим тэггировать имена людей в тексте. Для этого берем корпус имен (nltk.corpus.names), создаем класс тэггера, задаем логику: если слово входит в множество имен, присваиваем ему тэг имени "NNP", иначе тэг "None".

In [15]:
nltk.download('names')

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [16]:
from nltk.tag import SequentialBackoffTagger
from nltk.corpus import names

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in names.words()])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'NNP'
        else:
             return None
            
nt = NamesTagger()
print(nt.tag(['Katya'])) 
print(nt.tag(['Adam'])) 
print(nt.tag(['Window']))            

[('Katya', 'NNP')]
[('Adam', 'NNP')]
[('Window', None)]


Разметка предложений

На вход тэггеру также можно подавать предложения, self.tag() будет применяться к каждому слову предложения.

In [17]:
bigram_tagger.tag_sents([['make', 'America', 'great', 'again'], ['winter', 'is', 'coming']])

[[('make', 'VB'), ('America', 'NP-TL'), ('great', 'JJ'), ('again', 'RB')],
 [('winter', 'NN'), ('is', 'BEZ'), ('coming', 'VBG')]]

####  Разметка корпусов

Некоторые из корпусов, включенных в NLTK, были уже размечены. Вот пример того, что вы можете увидеть, если откроете nltk.corpus.brown

In [18]:
nltk.corpus.brown.tagged_words()

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]

In [19]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [20]:
nltk.corpus.brown.tagged_words(tagset='universal')

[('The', 'DET'), ('Fulton', 'NOUN'), ...]

Частота частей речи в корпусе

In [21]:
brown_news_tagged = nltk.corpus.brown.tagged_words(categories='adventure', tagset='universal')
tag_fd = nltk.FreqDist(tag for (word, tag) in brown_news_tagged)
tag_fd.most_common()

[('NOUN', 13354),
 ('VERB', 12274),
 ('.', 10929),
 ('DET', 8155),
 ('ADP', 7069),
 ('PRON', 5205),
 ('ADV', 3879),
 ('ADJ', 3364),
 ('PRT', 2436),
 ('CONJ', 2173),
 ('NUM', 466),
 ('X', 38)]

In [22]:
!pip install pyconll

In [23]:
import pyconll

In [24]:
!curl -O https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!curl -O https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0 77.2M    0 65808    0     0  16452      0  1:22:05  0:00:04  1:22:01 16321
  8 77.2M    8 6836k    0     0  1367k      0  0:00:57  0:00:05  0:00:52 1379k
 18 77.2M   18 13.9M    0     0  2384k      0  0:00:33  0:00:06  0:00:27 2982k
 23 77.2M   23 17.9M    0     0  2624k      0  0:00:30  0:00:07  0:00:23 3780k
 33 77.2M   33 26.1M    0     0  3342k      0  0:00:23  0:00:08  0:00:15 5591k
 40 77.2M   40 31.2M    0     0  3559k      0  0:00

In [25]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [26]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [27]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [28]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [29]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [30]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)

In [31]:
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', None),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', None),
 ('к', 'ADP'),
 ('ещё', None),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.6952111347015806

In [32]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [34]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [35]:
test_enc_labels = le.transform(test_label)

In [36]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [37]:
hvectorizer = HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100)

In [38]:
X_train = hvectorizer.fit_transform(train_tok)

In [39]:
X_test = hvectorizer.transform(test_tok)

In [40]:
X_train.shape

(871526, 100)

In [41]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)

LogisticRegression(random_state=0)

In [42]:
pred = lr.predict(X_test)

In [43]:
accuracy_score(test_enc_labels, pred)

0.686221480807468

###  Извлечение именованных сущностей (NER) и отношений

b-person
i-person
i-person
i-person
o-person

In [44]:
# person
# no_person

Одна из самых популярных задач NLP – извлечении именованных сущностей (Named-entity recognition, NER). Задача NER – выделить спаны сущностей в тексте (спан – непрерывный фрагмент текста). Допустим, есть новостной текст, и мы хотим выделить в нем сущности (некоторый заранее зафиксированный набор — например, персоны, локации, организации, даты и так далее). Задача NER – понять, что участок текста “1 января 1997 года” является датой, “Кофи Аннан” – персоной, а “ООН” – организацией.

##### Зачем?
1.  Cтруктуризация неструктурированных данных. Пусть у вас есть какой-то текст (или набор текстов), и данные из него нужно ввести в базу данных (таблицу). 
2. Шаг в сторону “понимания” текста. Это может как иметь самостоятельную ценность, так и помочь лучше решать другие задачи NLP. Без решения задачи NER тяжело представить себе решение многих задач NLP, допустим, разрешение местоименной анафоры или построение вопросно-ответных систем. 

Пример 1: Местоименная анафора позволяет нам понять, к какому элементу текста относится местоимение. Например, пусть мы хотим проанализировать текст “Прискакал Чарминг на белом коне. Принцесса выбежала ему навстречу и поцеловала его”. Если мы выделили на слове “Чарминг” сущность Персона, то машина сможет намного легче понять, что принцесса, скорее всего, поцеловала не коня, а принца Чарминга.

Пример 2: Теперь приведем пример, как выделение именованных сущностей может помочь при построении вопросно-ответных систем. Если задать в вашем любимом поисковике вопрос «Кто играл роль Дарта Вейдера в фильме “Империя наносит ответный удар”», то с большой вероятностью вы получите верный ответ. Это делается как раз с помощью выделения именованных сущностей: выделяем сущности (фильм, роль и т. п.), понимаем, что нас спрашивают, и дальше ищем ответ в базе данных.

Для решения NER-задач существуют множество инструментов. Рассмотрим несколько из них.


#### NLTK

<img src='image/nltk.PNG'>

Для разметки NER с помощью NLTK сначала производим токенизацию слов, затем POS тэггинг. 
В качестве документа возьмем статью 'https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news, распарсим ее с помощью BeautifulSoup.

In [45]:
!pip install html5lib

In [46]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [47]:
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')

nltk.pos_tag(nltk.word_tokenize(document))

[('F.B.I', 'NNP'),
 ('.', '.'),
 ('Agent', 'NNP'),
 ('Peter', 'NNP'),
 ('Strzok', 'NNP'),
 (',', ','),
 ('Who', 'NNP'),
 ('Criticized', 'NNP'),
 ('Trump', 'NNP'),
 ('in', 'IN'),
 ('Texts', 'NNP'),
 (',', ','),
 ('Is', 'NNP'),
 ('Fired', 'NNP'),
 ('-', ':'),
 ('The', 'DT'),
 ('New', 'NNP'),
 ('York', 'NNP'),
 ('Times', 'NNP'),
 ('SectionsSEARCHSkip', 'NNP'),
 ('to', 'TO'),
 ('contentSkip', 'VB'),
 ('to', 'TO'),
 ('site', 'NN'),
 ('indexPoliticsToday', 'JJ'),
 ('’', 'NNP'),
 ('s', 'NN'),
 ('PaperPolitics|F.B.I', 'NNP'),
 ('.', '.'),
 ('Agent', 'NNP'),
 ('Peter', 'NNP'),
 ('Strzok', 'NNP'),
 (',', ','),
 ('Who', 'NNP'),
 ('Criticized', 'NNP'),
 ('Trump', 'NNP'),
 ('in', 'IN'),
 ('Texts', 'NNP'),
 (',', ','),
 ('Is', 'VBZ'),
 ('Firedhttps', 'NNP'),
 (':', ':'),
 ('//www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.htmlAdvertisementContinue',
  'JJ'),
 ('reading', 'VBG'),
 ('the', 'DT'),
 ('main', 'JJ'),
 ('storySupported', 'JJ'),
 ('byContinue', 'NN'),
 ('reading', 'VBG'),
 ('

С помощью функции nltk.ne_chunk () мы можем распознавать именованные сущности с помощью классификатора, который добавляет метки категорий, такие как PERSON, ORGANIZATION и GPE.

In [48]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [49]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [50]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Agent Peter Strzok', 'PERSON'),
 ('Andrew G. McCabe', 'PERSON'),
 ('Anthony D. Weiner', 'PERSON'),
 ('Army', 'ORGANIZATION'),
 ('China', 'GPE'),
 ('Christopher', 'PERSON'),
 ('Clinton', 'PERSON'),
 ('CompanyNYTCoContact', 'ORGANIZATION'),
 ('Congress', 'ORGANIZATION'),
 ('David Bowdich', 'PERSON'),
 ('Director Wray', 'PERSON'),
 ('F.B.I.', 'ORGANIZATION'),
 ('Georgetown University', 'ORGANIZATION'),
 ('Goelman', 'PERSON'),
 ('Hillary', 'ORGANIZATION'),
 ('Hillary Clinton', 'PERSON'),
 ('Horowitz', 'PERSON'),
 ('House', 'ORGANIZATION'),
 ('Hundreds', 'PERSON'),
 ('IndexSite Information', 'ORGANIZATION'),
 ('Is', 'PERSON'),
 ('Is Fired', 'PERSON'),
 ('James B. Comey', 'PERSON'),
 ('Lisa Page', 'PERSON'),
 ('March', 'GPE'),
 ('Michael E. Horowitz', 'PERSON'),
 ('Michael S. SchmidtAug', 'PERSON'),
 ('Midyear Exam', 'PERSON'),
 ('Mr.', 'PERSON'),
 ('Mr. Bowdich', 'PERSON'),
 ('Mr. Goelman', 'PERSON'),
 ('Mr. Horowitz', 'PERSON'),
 ('Mr. McCabe', 'PERSON'),
 ('Mr. Mueller', 'PERSON'),
 ('

#### Spacy

Spacy значительно быстрее NLTK, так как она написана на Cython и работает с объектами.
Для NER Spacy работает как простой классификатор (неглубокая нейронная сеть с одним скрытым слоем). Объект Doc хранит последовательности токенов и все их аннотации.

<img src='image/spacy.PNG'>

In [51]:
!pip -q install spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm

2021-09-18 17:43:03.993981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-09-18 17:43:03.994023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0-py3-none-any.whl (13.6 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-09-18 17:43:25.580489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-09-18 17:43:25.580538: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [52]:
#!pip install -U spacy
#!python -m spacy info
import spacy
from spacy import displacy
# import en_core_web_md
nlp = spacy.load('en_core_web_sm')
# nlp = en_core_web_md.load()
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')


#### Deeppavlov

DeepPavlov-это библиотека ИИ с открытым исходным кодом, построенная на TensorFlow и Keras. DeepPavlov предназначена для разработки чат-ботов и сложных разговорных систем, исследований в области nlp и, в частности, диалоговых систем.

DeepPavlov использует несколько более новый вариант глубокой нейронной архитектуры Flair, известный как гибридная модель Bi-LSTM-CRF.

<img src='image/deeppavlov.PNG'>

Используем предтренированную модель "ner_ontonotes_bert_mult", которая работает с разными языками, в том числе и русским. Подадим на распознование предложение на русском языке. 

In [56]:
!pip install deeppavlov

  Using cached deeppavlov-0.17.0-py3-none-any.whl (928 kB)
  Using cached aio_pika-6.4.1-py3-none-any.whl (40 kB)
  Using cached fastapi-0.47.1-py3-none-any.whl (43 kB)
  Using cached uvloop-0.14.0.tar.gz (2.0 MB)
  Using cached deeppavlov-0.16.0-py3-none-any.whl (901 kB)
  Using cached deeppavlov-0.15.0-py3-none-any.whl (907 kB)
  Using cached deeppavlov-0.14.1-py3-none-any.whl (988 kB)
  Using cached deeppavlov-0.14.0-py3-none-any.whl (988 kB)
  Using cached tqdm-4.41.1-py2.py3-none-any.whl (56 kB)
  Using cached pandas-0.25.3-cp38-cp38-win_amd64.whl (9.4 MB)
  Using cached nltk-3.4.5-py3-none-any.whl
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached pyOpenSSL-19.1.0-py2.py3-none-any.whl (53 kB)

    ERROR: Command errored out with exit status 1:
     command: 'c:\users\ifl\documents\github\.venv\scripts\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Ifl\\AppData\\Local\\Temp\\pip-install-d6zfc3yq\\uvloop_708508fab4d84f09897c8cc94835e70e\\setup.py'"'"'; __file__='"'"'C:\\Users\\Ifl\\AppData\\Local\\Temp\\pip-install-d6zfc3yq\\uvloop_708508fab4d84f09897c8cc94835e70e\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Ifl\AppData\Local\Temp\pip-pip-egg-info-_h235iro'
         cwd: C:\Users\Ifl\AppData\Local\Temp\pip-install-d6zfc3yq\uvloop_708508fab4d84f09897c8cc94835e70e\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      Fi


  Using cached scipy-1.4.1-cp38-cp38-win_amd64.whl (31.0 MB)
  Using cached rusenttokenize-0.0.5-py3-none-any.whl (10 kB)
  Using cached scikit-learn-0.21.2.tar.gz (12.2 MB)
  Using cached uvicorn-0.11.7-py3-none-any.whl (43 kB)
  Using cached pyTelegramBotAPI-3.6.7-py3-none-any.whl
  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
  Using cached overrides-2.7.0-py3-none-any.whl
  Using cached requests-2.22.0-py2.py3-none-any.whl (57 kB)
  Using cached sacremoses-0.0.35-py3-none-any.whl
  Using cached pytz-2019.1-py2.py3-none-any.whl (510 kB)
  Using cached pydantic-1.3-py36.py37.py38-none-any.whl (85 kB)
  Using cached Cython-0.29.14-cp38-cp38-win_amd64.whl (1.7 MB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached prometheus_client-0.7.1-py3-none-any.whl
  Using cached h5py-2.10.0-cp38-cp38-win_amd64.whl (2.5 MB)
  Using cached ruamel.yaml-0.15.100-cp38-cp38-win_amd64.whl
  Using cached aiormq-3.3.1-py3-none-any.whl (28 kB)
  U


  Could not locate executable flang
  don't know how to compile Fortran code on platform 'nt'
    NOT AVAILABLE
  
  atlas_3_10_blas_threads_info:
  Setting PTATLAS=ATLAS
    libraries tatlas not found in ['c:\\users\\ifl\\documents\\github\\.venv\\lib', 'C:\\']
    NOT AVAILABLE
  
  atlas_3_10_blas_info:
    libraries satlas not found in ['c:\\users\\ifl\\documents\\github\\.venv\\lib', 'C:\\']
    NOT AVAILABLE
  
  atlas_blas_threads_info:
  Setting PTATLAS=ATLAS
    libraries ptf77blas,ptcblas,atlas not found in ['c:\\users\\ifl\\documents\\github\\.venv\\lib', 'C:\\']
    NOT AVAILABLE
  
  atlas_blas_info:
    libraries f77blas,cblas,atlas not found in ['c:\\users\\ifl\\documents\\github\\.venv\\lib', 'C:\\']
    NOT AVAILABLE
  
  accelerate_info:
    NOT AVAILABLE
  
  c:\users\ifl\documents\github\.venv\lib\site-packages\numpy\distutils\system_info.py:1896: UserWarning:
      Optimized (vendor) Blas libraries are not found.
      Falls back to netlib Blas library which has w

In [54]:
# !python -m venv env 
# #.\env\Scripts\activate.bat
# !pip install deeppavlov
# !python -m deeppavlov install squad_bert

#!python -m deeppavlov install ner_ontonotes
import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])


  copying sklearn\compose\tests\test_target.py -> build\lib.win-amd64-3.8\sklearn\compose/tests
  copying sklearn\compose\tests\__init__.py -> build\lib.win-amd64-3.8\sklearn\compose/tests
  creating build\lib.win-amd64-3.8\sklearn\covariance
  copying sklearn\covariance\elliptic_envelope.py -> build\lib.win-amd64-3.8\sklearn\covariance
  copying sklearn\covariance\empirical_covariance_.py -> build\lib.win-amd64-3.8\sklearn\covariance
  copying sklearn\covariance\graph_lasso_.py -> build\lib.win-amd64-3.8\sklearn\covariance
  copying sklearn\covariance\robust_covariance.py -> build\lib.win-amd64-3.8\sklearn\covariance
  copying sklearn\covariance\shrunk_covariance_.py -> build\lib.win-amd64-3.8\sklearn\covariance
  copying sklearn\covariance\__init__.py -> build\lib.win-amd64-3.8\sklearn\covariance
  creating build\lib.win-amd64-3.8\sklearn\covariance\tests
  copying sklearn\covariance\tests\test_covariance.py -> build\lib.win-amd64-3.8\sklearn\covariance/tests
  copying sklearn\covar

ModuleNotFoundError: No module named 'deeppavlov'

### Извлечение отношений

Можно извлекать не только именнованные сущности (NER), но и отношения между словами в предложении (Relation extraction).

#### Spacy

В уже знакомой нам библеотеке Spacy используем встроенные displaCy визуализатор со style="dep" для отображения отношений.

In [60]:
import spacy
from spacy import displacy
#install spacy model
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz
# import en_core_web_md
nlp = spacy.load("en_core_web_md")
doc = nlp("I think Barack Obama met founder of Facebook at occasion of a release of a new NLP algorithm.")

displacy.render(doc, style="dep") # (1)
displacy.render(doc, style="ent") # (2)

  Using cached pydantic-1.8.2-cp38-cp38-win_amd64.whl (2.0 MB)
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-py3-none-any.whl size=98072931 sha256=f69826741a15365961e74e8d9685c40087440afa63d59aa39ca6c59635d0af2a
  Stored in directory: c:\users\ifl\appdata\local\pip\cache\wheels\31\1f\21\52bc46cea1e50c10a3b246d5d266455b9a0e77c380bb7ac928
Successfully built en-core-web-md
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.3
    Uninstalling pydantic-1.3:
      Successfully uninstalled pydantic-1.3


OSError: [E053] Could not read config.cfg from c:\users\ifl\documents\github\.venv\lib\site-packages\en_core_web_md\en_core_web_md-2.2.0\config.cfg

#### NLTK

Распарсим документ nltk.corpus.ieer.parsed_docs('NYT_19980315'), extract_rels позволяет нам выделять необходимые отношения, в данном случае мы ищем пары сущностей <'ORG', 'LOC'>. Также мы указали, что текст должен подходить под регулярное выражение IN.

In [62]:
nltk.download('ieer')

[nltk_data] Downloading package ieer to
[nltk_data]     C:\Users\Ifl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\ieer.zip.


True

In [63]:
import re
import nltk
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
     for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
                                      corpus='ieer', pattern = IN):
            print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


#### Allennlp

Попробовать извлекать отношения онлайн можно с помощью демо Allennlp  https://demo.allennlp.org/dependency-parsing/. 
Allen NLP предлагает две модели NER с различными архитектурами: Gated Recurrent Unit (GRU) Network, bi-LSTM-CRF model.

<img src='image/allennlp.PNG'>

In [65]:
!pip install pymorphy2

  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)


In [66]:
from pymorphy2 import MorphAnalyzer

In [67]:
morpher = MorphAnalyzer()

In [68]:
morpher.parse("кошка")[0]

Parse(word='кошка', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='кошка', score=0.9375, methods_stack=((DictionaryAnalyzer(), 'кошка', 134, 0),))

In [72]:
!pip install natasha

  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=64f70a9bb051f95373e1f7e5fafdb49e64ad6681ee7b8f9ea17d64650e5c2fdd
  Stored in directory: c:\users\ifl\appdata\local\pip\cache\wheels\45\23\de\5789a92962483fd33cb06674792b9697c1b3766d7c7742830e
Successfully built intervaltree


In [73]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [74]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '
doc = Doc(text)

In [75]:
doc.segment(segmenter)
display(doc.tokens[:5])
display(doc.sents[:5])

[DocToken(stop=5, text='Посол'),
 DocToken(start=6, stop=13, text='Израиля'),
 DocToken(start=14, stop=16, text='на'),
 DocToken(start=17, stop=24, text='Украине'),
 DocToken(start=25, stop=30, text='Йоэль')]

[DocSent(stop=218, text='Посол Израиля на Украине Йоэль Лион признался, чт..., tokens=[...]),
 DocSent(start=219, stop=257, text='Свое заявление он разместил в Twitter.', tokens=[...]),
 DocSent(start=258, stop=424, text='«Я не могу понять, как прославление тех, кто непо..., tokens=[...]),
 DocSent(start=425, stop=592, text='Украина не должна забывать о преступлениях, совер..., tokens=[...]),
 DocSent(start=593, stop=798, text='11 декабря Львовский областной совет принял решен..., tokens=[...])]

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

In [78]:
!pip install corus

In [79]:
import corus

In [81]:
!curl -O http://www.labinform.ru/pub/named_entities/collection5.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1 1855k    1 34169    0     0  34169      0  0:00:55 --:--:--  0:00:55  101k
  8 1855k    8  150k    0     0   150k      0  0:00:12  0:00:01  0:00:11  113k
 15 1855k   15  279k    0     0   139k      0  0:00:13  0:00:02  0:00:11  120k
 21 1855k   21  399k    0     0   133k      0  0:00:13  0:00:03  0:00:10  119k
 27 1855k   27  513k    0     0   128k      0  0:00:14  0:00:04  0:00:10  118k
 34 1855k   34  641k    0     0   128k      0  0:00:14  0:00:05  0:00:09  120k
 41 1855k   41  761k    0     0   126k      0  0:00:14  0:00:06  0:00:08  122k
 48 1855k   48  890k    0     0   127k      0  0:00:14  0:00:07  0:00:07  122k
 54 1855k   54 1002k    0     0   125k      0  0:00:14  0:00:08  0:00:06  117k
 59 1855k   59 1106k    0     0   122k      0  0:00

In [82]:
!unzip collection5.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [94]:
import zipfile
with zipfile.ZipFile('collection5.zip', 'r') as zip_f:
    zip_f.extractall()

In [87]:
!pip install wget
import wget

In [88]:
wget.download('http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip')

100% [..........................................................................] 3363777 / 3363777

'Persons-1000.zip'

In [93]:
import zipfile
with zipfile.ZipFile('Persons-1000.zip', 'r') as zip_f:
    zip_f.extractall()

In [96]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 5: character maps to <undefined>

In [97]:
path = 'Persons-1000.zip'

records = corus.persons.load_persons(path)
rec = next(records)

In [98]:
rec

PersonsMarkup(
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[PersonsSpan(
         id=1,
         start=308,
         stop=324,
         value='ГРИГОРИЙ КАР

In [99]:
# !pip install razdel

In [100]:
from razdel import tokenize

In [101]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

AttributeError: 'PersonsSpan' object has no attribute 'type'

In [102]:
import pandas as pd

In [103]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [104]:
df_words['tag'].value_counts()

Series([], Name: tag, dtype: int64)

In [105]:
df_words.head(3)

,word,tag


In [106]:
df_words.shape

(0, 2)

In [107]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [108]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
train_x.apply(len).max(axis=0)

In [ ]:
valid_x

In [ ]:
# char level
#train_x = train_x.apply(lambda x: ' '.join(list(x)))
#valid_x = valid_x.apply(lambda x: ' '.join(list(x)))

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [ ]:
len(vectorize_layer.get_vocabulary())

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
mmodel = modelNER()

In [ ]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)